### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [32]:
#Dependencies and Setup
from pathlib import Path
import pandas as pd
import numpy as np

#Raw data file
data_folder_in = (Path("/01_Data_Analytics/Git_Clones/UTAUS201807DATA2/\
homework-instructions/04-Numpy-Pandas/Instructions/HeroesOfPymoli/Resources/"))
file_to_open = data_folder_in / "purchase_data.csv"

#Read purchasing file and store into pandas data frame
df_reader = pd.read_csv(file_to_open)

## Player Count

* Display the total number of players


In [33]:
player_count = df_reader['SN'].nunique()

player_count

576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [34]:
#pull summary data from existing dataframe
unique_item = df_reader['Item ID'].nunique()
avg_pp = df_reader['Price'].mean()
tot_pur = df_reader['Item Name'].count()
tot_rev = df_reader['Price'].sum()

#create new dataframe
df_sum = pd.DataFrame({
                    'Number of Unique Items':[unique_item],
                    'Average Price':[avg_pp],
                    'Number of Purchases':[tot_pur],
                    'Total Revenue':[tot_rev]
                    })

#add formatting to columns
df_sum['Average Price'] = df_sum['Average Price'].map('${:,.2f}'.format)
df_sum['Total Revenue'] = df_sum['Total Revenue'].map('${:,.2f}'.format)

df_sum

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [35]:
#groupy by on Gender and SN
player_sum = df_reader.groupby(['Gender','SN']).count().reset_index()

#assing gender counts to variables
male_count = player_sum['Gender'].value_counts()['Male']
female_count = player_sum['Gender'].value_counts()['Female']
other_count = player_sum['Gender'].value_counts()['Other / Non-Disclosed']

#calculate gender percentages
male_per = male_count / player_count
female_per = female_count / player_count
other_per = other_count / player_count

#create new dataframe
df_gdem = pd.DataFrame({
                        'Gender':['Male','Female','Other / Non-Disclosed'],
                        'Percentage of Players':[male_per, female_per, other_per],
                        'Total Count':[male_count, female_count, other_count]
                        })

#add formatting
df_gdem['Percentage of Players'] = df_gdem['Percentage of Players'].map('{:,.0%}'.format)

df_gdem

,Gender,Percentage of Players,Total Count
0,Male,84%,484
1,Female,14%,81
2,Other / Non-Disclosed,2%,11



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [36]:
#create new groupby dataframe with formula specific columns
g = {'Gender':['count'],'Price':['mean','sum']} #create dictionary with formula by column
by_gender = df_reader.groupby('Gender').agg(g).reset_index() #use agg function to apply dictionary to DF
by_gender.columns = by_gender.columns.droplevel(0) #reset columns levels
by_gender.columns = ['Gender','Purchase Count', 'Average Purchase Price','Total Purchase Value'] #rename columns

#merge with gender demographics to calculate Avg Purchase Total per Person'
by_gender = by_gender.merge(df_gdem, on = 'Gender')
by_gender['Avg Purchase Total per Person'] = by_gender['Total Purchase Value'] / by_gender['Total Count']

#create a DF copy to avoid indexing warnings and drop unnecesary columns
gen_sum = by_gender.copy()
gen_sum = gen_sum.drop(columns = ['Percentage of Players', 'Total Count'])

#add formatting
gen_sum['Average Purchase Price'] = gen_sum['Average Purchase Price'].map('${:,.2f}'.format)
gen_sum['Total Purchase Value'] = gen_sum['Total Purchase Value'].map('${:,.2f}'.format)
gen_sum['Avg Purchase Total per Person'] = gen_sum['Avg Purchase Total per Person'].map('${:,.2f}'.format)

gen_sum

,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Purchase Total per Person
0,Female,113,$3.20,$361.94,$4.47
1,Male,652,$3.02,"$1,967.64",$4.07
2,Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [37]:
df_demo = df_reader.copy()

#Establish bins and labels for ages
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 99999]
grp_labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

#cut bins and labels into dataframe and get count of unique players
df_demo['Age Summary'] = pd.cut(df_demo['Age'], bins, labels = grp_labels)
by_demo_sum = df_demo.drop(columns = ['Purchase ID', 'Gender', 'Item ID', 'Item Name', 'Price']) #drop unnecessary columns
by_demo_sum = by_demo_sum.drop_duplicates() #drop duplicate rows
by_demo_sum = by_demo_sum.groupby('Age Summary').count()['SN'].reset_index()

#rename columns and calculate percentages
by_demo_sum.columns = ['Age Summary', 'Total Count']
by_demo_sum['Percentage of Players'] = by_demo_sum['Total Count']/by_demo_sum['Total Count'].sum()

#add formatting
by_demo_sum['Percentage of Players'] = by_demo_sum['Percentage of Players'].map('{:,.0%}'.format)

by_demo_sum

,Age Summary,Total Count,Percentage of Players
0,<10,17,3%
1,10-14,22,4%
2,15-19,107,19%
3,20-24,258,45%
4,25-29,77,13%
5,30-34,52,9%
6,35-39,31,5%
7,40+,12,2%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [38]:
#create new groupby dataframe with formula specific columns
d = {'Age Summary':['count'],'Price':['mean','sum']} #create dictionary with formula by column
by_demo_data = df_demo.groupby('Age Summary').agg(d).reset_index() #use agg function to apply dictionary to DF
by_demo_data.columns = by_demo_data.columns.droplevel(0) #reset columns levels
by_demo_data.columns = ['Age Summary', 'Purchase Count', 'Average Purchase Price', 'Total Purchase Value'] #rename columns

#merge with demographic summary data to calculate Avg Purchase Total per Person'
by_demo_data = by_demo_data.merge(by_demo_sum, on = 'Age Summary', how = 'left')
by_demo_data['Average Purchase Total per Person'] = by_demo_data['Total Purchase Value']/by_demo_data['Total Count']

#create a DF copy to avoid indexing warnings and drop unnecesary columns
by_demo = by_demo_data.copy()
by_demo = by_demo.drop(columns = ['Total Count', 'Percentage of Players'])

#add formatting
by_demo['Average Purchase Price'] = by_demo['Average Purchase Price'].map('${:,.2f}'.format)
by_demo['Total Purchase Value'] = by_demo['Total Purchase Value'].map('${:,.2f}'.format)
by_demo['Average Purchase Total per Person'] = by_demo['Average Purchase Total per Person'].map('${:,.2f}'.format)

by_demo

,Age Summary,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
0,<10,23,$3.35,$77.13,$4.54
1,10-14,28,$2.96,$82.78,$3.76
2,15-19,136,$3.04,$412.89,$3.86
3,20-24,365,$3.05,"$1,114.06",$4.32
4,25-29,101,$2.90,$293.00,$3.81
5,30-34,73,$2.93,$214.00,$4.12
6,35-39,41,$3.60,$147.67,$4.76
7,40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [39]:
#create new groupby dataframe with formula specific columns
s = {'SN':['count'],'Price':['mean', 'sum']} #create dictionary with formula by column
top_5spend = df_reader.groupby('SN').agg(s).nlargest(5,[('Price','sum')]).reset_index() #use agg function to apply dictionary to DF
top_5spend.columns = top_5spend.columns.droplevel(0) #reset columns levels
top_5spend.columns = ['SN', 'Purchase Count', 'Average Purchase Price', 'Total Purchase Value'] #rename columns

#add formatting
top_5spend['Average Purchase Price'] = top_5spend['Average Purchase Price'].map('${:,.2f}'.format)
top_5spend['Total Purchase Value'] = top_5spend['Total Purchase Value'].map('${:,.2f}'.format)

top_5spend

,SN,Purchase Count,Average Purchase Price,Total Purchase Value
0,Lisosia93,5,$3.79,$18.96
1,Idastidru52,4,$3.86,$15.45
2,Chamjask73,3,$4.61,$13.83
3,Iral74,4,$3.40,$13.62
4,Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [40]:
#create new groupby dataframe with formula specific columns
pop = {'Item ID':['count'],'Price':['mean', 'sum']} #create dictionary with formula by column
top_5pop = df_reader.groupby(['Item ID','Item Name']).agg(pop).nlargest(5,[('Item ID','count')]).reset_index() #use agg function to apply dictionary to DF
top_5pop.columns = top_5pop.columns.droplevel(0) #reset columns levels
top_5pop.columns = ['Item ID', 'Item Name', 'Purchase Count', 'Item Price', 'Total Purchase Value'] #rename columns
top_5pop = top_5pop.sort_values(by=['Purchase Count', 'Item ID'], ascending = False) #sort by Purchase Count then Item ID

#add formatting
top_5pop['Item Price'] = top_5pop['Item Price'].map('${:,.2f}'.format)
top_5pop['Total Purchase Value'] = top_5pop['Total Purchase Value'].map('${:,.2f}'.format)

top_5pop

,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
0,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
3,145,Fiery Glass Crusader,9,$4.58,$41.22
2,108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
1,82,Nirvana,9,$4.90,$44.10
4,19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [41]:
#create new groupby dataframe with formula specific columns
pro = {'Item ID':['count'],'Price':['mean', 'sum']} #create dictionary with formula by column
top_5pro = df_reader.groupby(['Item ID','Item Name']).agg(pro).nlargest(5,[('Price','sum')]).reset_index() #use agg function to apply dictionary to DF
top_5pro.columns = top_5pro.columns.droplevel(0) #reset columns levels
top_5pro.columns = ['Item ID', 'Item Name', 'Purchase Count', 'Item Price', 'Total Purchase Value'] #rename columns

#add formatting
top_5pro['Item Price'] = top_5pro['Item Price'].map('${:,.2f}'.format)
top_5pro['Total Purchase Value'] = top_5pro['Total Purchase Value'].map('${:,.2f}'.format)

top_5pro

,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
0,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
1,82,Nirvana,9,$4.90,$44.10
2,145,Fiery Glass Crusader,9,$4.58,$41.22
3,92,Final Critic,8,$4.88,$39.04
4,103,Singed Scalpel,8,$4.35,$34.80


## Observable Trends

* 1) Males over the age of 15 spend the most on games and account for ~75% of revenue.

* 2) Females, age 15 to 24, generate 10% of revenue and there are only 3 games that they purchase more than males. This is an opportunity to investigate further to help grow the female market.

* 4) Despite the label, the last table does not determine the profitability, but rather the revenue. The data needs cost to determine profitability.